In [ ]:
!pip install sentence-transformers

In [24]:
# Change the path of source file and Model Embeddings

In [ ]:
# from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import re
import h5py
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

ps = PorterStemmer()

In [ ]:
nltk.download("stopwords")
stopwords = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Load the Data
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/product_names.csv', engine='python')
df.shape

(2397873, 1)

In [ ]:
df['lower_names'] = df['Product Name'].str.lower().str.strip()
df = df.drop_duplicates(subset='lower_names')
df.shape

(2394761, 2)

In [ ]:
df[:10]

,Product Name,lower_names
0,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...","all-new fire hd 8 tablet, 8 hd display, wi-fi,..."
1,Kindle Oasis E-reader with Leather Charging Co...,kindle oasis e-reader with leather charging co...
2,"Amazon Kindle Lighted Leather Cover,,,\r\nAmaz...","amazon kindle lighted leather cover,,,\r\namaz..."
3,"Amazon Kindle Lighted Leather Cover,,,\r\nKind...","amazon kindle lighted leather cover,,,\r\nkind..."
4,"Kindle Keyboard,,,\r\nKindle Keyboard,,,","kindle keyboard,,,\r\nkindle keyboard,,,"
5,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...","all-new fire hd 8 tablet, 8 hd display, wi-fi,..."
6,"Fire HD 8 Tablet with Alexa, 8 HD Display, 32 ...","fire hd 8 tablet with alexa, 8 hd display, 32 ..."
7,Amazon 5W USB Official OEM Charger and Power A...,amazon 5w usb official oem charger and power a...
8,"All-New Kindle E-reader - Black, 6 Glare-Free ...","all-new kindle e-reader - black, 6 glare-free ..."
9,"Amazon Kindle Fire Hd (3rd Generation) 8gb,,,\...","amazon kindle fire hd (3rd generation) 8gb,,,\..."


In [ ]:
def data_preprocessor(text):
  # Convert the text to lower for Consistency
  text = text.lower()
  # Remove sentence breaks
  text = re.sub(r'\r\n', '', text)
  # Remove unwanted commas
  text = re.sub(r',', '', text)
  text = re.sub(r'-', ' ', text)
  tokens = text.split(" ")
  # Remove stop words and stem the words
  tokens = [ps.stem(t) for t in tokens if t not in stopwords]
  text = " ".join(tokens)
  return text

In [ ]:
data_preprocessor("Rajesh is from Chennai and He speaks fluent Tamil")

'rajesh chennai speak fluent tamil'

In [ ]:
preprocessed_data = df['Product Name'][:400000].apply(data_preprocessor).tolist()

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-MiniLM-L6-v2', device='cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
tokenized_input = model.tokenize(preprocessed_data) # FIXME: Unable to offload this task to GPU

# Forward pass through the model to obtain embeddings
embeddings = model.encode(preprocessed_data, convert_to_tensor=True, show_progress_bar=True)

In [ ]:
def store_embeddings(embeddings):
  # Save to HDF5
  with h5py.File('/content/drive/MyDrive/Colab Notebooks/product_embeddings_overall_para.h5', 'w') as hf:
      hf.create_dataset('embeddings', data=embeddings)

In [ ]:
def get_inference(search_query, embed, training=False):
  if training:
    embed = embed.cpu()
  search_embed = model.encode([data_preprocessor(search_query)])[0]
  similarities = cosine_similarity([search_embed], embed)[0]

  top_indices = similarities.argsort()[-10:][::-1]
  suggestions = df.iloc[top_indices]['Product Name'].tolist()
  return suggestions

In [ ]:
import h5py
# Using context manager creatting issue like "Invalid Dataset Identifier"
mf = h5py.File('/content/drive/MyDrive/Colab Notebooks/product_embeddings_overall_para.h5', 'r')
embeds = mf['embeddings']

get_inference("fire tv", embeds)

['Amazon Fire Tv,,,\r\nAmazon Fire Tv,,,',
 'Fire TV Stick Streaming Media Player Pair Kit',
 'Playmobil 5104 Fire Guardian with LED Fire',
 'Fire Engine',
 'Amazon Fire TV Gaming Edition Streaming Media Player',
 'WatchGuard Firebox T35-W Network Security/Firewall Appliance',
 'Logitech TV Cam for Revue with Google TV',
 'Bburago 1:32 Street Fire Kit Jaguar XF',
 'Walking Dead TV "Series 8 Rick" Action Figure',
 'Robocar Poli -Korean Made TV Animation Toy-Fire Engine- Roi/Roy (Transformer)']

In [ ]:
get_inference("Fire TV", embeds)

['Amazon Fire Tv,,,\r\nAmazon Fire Tv,,,',
 'Fire TV Stick Streaming Media Player Pair Kit',
 'Playmobil 5104 Fire Guardian with LED Fire',
 'Fire Engine',
 'Amazon Fire TV Gaming Edition Streaming Media Player',
 'WatchGuard Firebox T35-W Network Security/Firewall Appliance',
 'Logitech TV Cam for Revue with Google TV',
 'Bburago 1:32 Street Fire Kit Jaguar XF',
 'Walking Dead TV "Series 8 Rick" Action Figure',
 'Robocar Poli -Korean Made TV Animation Toy-Fire Engine- Roi/Roy (Transformer)']

In [ ]:
get_inference("ereader", embeds)

['OOK',
 'EZE DOPE',
 'Ingear Front Shirred One Piece',
 'Eberjey Womens Heather Active The Shorts',
 'Edimax AR-7286WNA',
 'Hape All-in-one Easel',
 'Elomi Womens Amazonia Classic Brief',
 'Edimax AR-7182WNA',
 'Ereader for Sleeve Case Bag for 6 inch Ereader Tablet Protective Cover Pouch',
 'Ook, 12']

In [ ]:
get_inference("e reader", embeds)

['Vtech Storio V.Reader Animated E-Book Reader - Penguins',
 'C-Pen Reader',
 'C-Pen Exam Reader',
 'The sharper image luggage scale - High precision digital reader',
 'Out of Print Literary and Book-Themed Canvas Tote Carrying Bag for Book Lovers, Readers, and Bibliophiles',
 "Reebok Women's Linear Read Scoop",
 'Replacement Reader Assy. KABA SOLITAIRE710/710-II',
 "O'NEILL Women's Elaine Printed Short",
 'Melissa and Doug Chalk Activity Book',
 'KLOO Zoo Reader Cards - Reading Games for Children']

In [ ]:
get_inference("black", embeds)

['Black Shorts',
 'Skpat Suitcase, Black (Negro)',
 "ful Women's Luggage, Black",
 'Movom Suitcase, Black (Negro)',
 'Roncato Suitcase, Black (Negro)',
 'P.E NATION Leggings Black',
 'ABUS AB145/30 NEGRO Padlock, Black',
 'Cavalet Hand Luggage, Black',
 'MOVOM Luggage Set, Black (Negro)',
 'Skpat Luggage Set, Black (Negro)']

In [ ]:
get_inference("Tablet with Alexa", embeds)

['Amazon Fire HD 8 with Alexa (8" HD Display Tablet)',
 'Fire Tablet with Alexa, 7 Display, 16 GB, Magenta - with Special Offers',
 'AmazonBasics Multi-Angle Portable Stand for iPad Tablet, E-reader and Phone - Black',
 'Fire Tablet with Alexa, 7" Display, 16 GB, Magenta - with Special Offers',
 'Brookstone iConvert Scanner for iPad and iPad 2 Tablets',
 'Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Tangerine - with Special Offers,',
 'Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Tangerine - with Special Offers',
 'Fire Tablet with Alexa, 7 Display, 16 GB, Blue - with Special Offers',
 'Compatible Tablet Replacement LCD Screen for ipad Mini2 Model A1489 A1490 A1491 with Professional Tools(Black)',
 'Fire HD 8 Tablet with Alexa, 8 HD Display, 32 GB, Tangerine - with Special Offers']

In [22]:
get_inference("Kindle Oasis E-reader with Leather Charging Cover - Merlot, 6 High-Resolution Display (300 ppi), Wi-Fi - Includes Special Offers", embeds)

['Kindle Oasis E-reader with Leather Charging Cover - Merlot, 6 High-Resolution Display (300 ppi), Wi-Fi - Includes Special Offers',
 'Kindle Oasis E-reader with Leather Charging Cover - Merlot, 6 High-Resolution Display (300 ppi), Wi-Fi - Includes Special Offers,,',
 'Kindle Oasis E-reader with Leather Charging Cover - Black, 6" High-Resolution Display (300 ppi), Wi-Fi - Includes Special Offers',
 'Kindle Oasis E-reader with Leather Charging Cover - Black, 6 High-Resolution Display (300 ppi), Wi-Fi - Includes Special Offers,,',
 'Kindle Oasis E-reader with Leather Charging Cover - Black, 6 High-Resolution Display (300 ppi), Wi-Fi - Includes Special Offers',
 'Kindle Oasis E-reader with Leather Charging Cover - Walnut, 6 High-Resolution Display (300 ppi), Wi-Fi - Includes Special Offers',
 'Kindle Paperwhite E-reader - White, 6 High-Resolution Display (300 ppi) with Built-in Light, Wi-Fi - Includes Special Offers,,',
 'All-New Kindle Oasis E-reader - 7 High-Resolution Display (300 ppi)

In [23]:
get_inference("E-reader, 6 High-Resolution Display", embeds)

['Kindle Paperwhite E-reader - White, 6 High-Resolution Display (300 ppi) with Built-in Light, Wi-Fi - Includes Special Offers,,',
 'Kindle Oasis E-reader with Leather Charging Cover - Black, 6" High-Resolution Display (300 ppi), Wi-Fi - Includes Special Offers',
 'Kindle Voyage E-reader, 6 High-Resolution Display (300 ppi) with Adaptive Built-in Light, PagePress Sensors, Wi-Fi - Includes Special Offers',
 'Kindle Voyage E-reader, 6 High-Resolution Display (300 ppi) with Adaptive Built-in Light, PagePress Sensors, Wi-Fi - Includes Special Offers,',
 'All-New Kindle Oasis E-reader - 7 High-Resolution Display (300 ppi), Waterproof, Built-In Audible, 32 GB, Wi-Fi - Includes Special Offers',
 'Kindle Oasis E-reader with Leather Charging Cover - Black, 6 High-Resolution Display (300 ppi), Wi-Fi - Includes Special Offers',
 'Kindle Oasis E-reader with Leather Charging Cover - Black, 6 High-Resolution Display (300 ppi), Wi-Fi - Includes Special Offers,,',
 'All-New Kindle Oasis E-reader - 7 H